In [1]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta,timezone

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2023-11-12 06:30:32.953657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

2.12.0
1.4.0


In [22]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

# Load Configuration Data and Constant Variable

In [13]:
projectId="pongthorn"
#_model='binary_xgb_tf_model'   # the is resonable because unsseen and tune 70%
all_prediction=True
is_evaluation=True



_model="demo2_binary_rf_tf_model/"  # best 75% on unseen test
model_version=f'{_model}_demo_tNov23'

model_gs_path=f"gs://demo2-tf-incident-pongthorn/{_model}"


dataset_id="SMartML"

data_table="new2_incident"
prediction_table="new2_result_binary_prediction_incident"


# unusedCols_unseen=['id','severity_name','imported_at','open_to_close_hour']

unusedCols_unseen=['id','severity_name','imported_at','range_open_to_close_hour']


# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(f"Prediction at {prediction_datetime} for {today_str} ({today})")

print(model_gs_path)

print(f"Data: {data_table} and Prediction: {prediction_table}")

Prediction at 2023-11-12 06:33:20.169436+00:00 for 2023-11-12 (2023-11-12 00:00:00)
gs://demo2-tf-incident-pongthorn/demo2_binary_rf_tf_model/
Data: new2_incident and Prediction: new2_result_binary_prediction_incident


# Load Model

In [14]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-11-12 06:33:23.9313 UTC kernel.cc:1243] Loading model from path gs://demo2-tf-incident-pongthorn/demo2_binary_rf_tf_model/assets/ with prefix 898a63dcc5b24f14
[INFO 23-11-12 06:33:24.7024 UTC decision_forest.cc:660] Model loaded with 300 root(s), 135716 node(s), and 6 input feature(s).
[INFO 23-11-12 06:33:24.7033 UTC abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-11-12 06:33:24.7038 UTC kernel.cc:1075] Use fast generic engine


Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# BigQuery Configuration

In [15]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.{data_table}"
predictResult_table_id=f"{projectId}.{dataset_id}.{prediction_table}"
print(new_data_table_id)
print(predictResult_table_id)

pongthorn.SMartML.new2_incident
pongthorn.SMartML.new2_result_binary_prediction_incident


In [16]:
try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
    
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prediction_item_date", "DATE", mode="REQUIRED"),    
    bigquery.SchemaField("label_binary_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_binary_severity", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") ,
    bigquery.SchemaField("model_version", "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new2_result_binary_prediction_incident already exists.


# Load data to Make Prediction

In [17]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     WHERE DATE(imported_at) = '{today_str}'
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     order by imported_at
    """

print(sql)


query_result=client.query(sql)
df=query_result.to_dataframe()
if df.empty==True:
  print("no data to make prediction")  
  # return "no data to make prediction"
print(df.info())



    SELECT *  FROM `pongthorn.SMartML.new2_incident` 
     order by imported_at
    
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        67 non-null     Int64         
 1   severity_id               67 non-null     Int64         
 2   severity_name             67 non-null     object        
 3   sla                       67 non-null     object        
 4   product_type              67 non-null     object        
 5   brand                     67 non-null     object        
 6   service_type              67 non-null     object        
 7   incident_type             67 non-null     object        
 8   open_to_close_hour        67 non-null     float64       
 9   range_open_to_close_hour  67 non-null     object        
 10  imported_at               67 non-null     datetime64[ns]
dtype

# Build Unseen data by removing label and others

In [18]:
unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   severity_id         67 non-null     Int64  
 1   sla                 67 non-null     object 
 2   product_type        67 non-null     object 
 3   brand               67 non-null     object 
 4   service_type        67 non-null     object 
 5   incident_type       67 non-null     object 
 6   open_to_close_hour  67 non-null     float64
dtypes: Int64(1), float64(1), object(5)
memory usage: 3.9+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour
57,2,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,5.750000
58,2,24x7 4Hrs Resolution Time,Server,HPE,Incident,Hard Disk Drive Failure,5.750000
59,2,24x7 6Hrs Response Time,Server,HPE,Incident,Hard Disk Drive Failure,21.066667
60,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,35.000000
61,3,24x7 4Hrs Response Time,Storage,NetApp,Incident,Hard Disk Drive Failure,7.633333
62,4,24x7 4Hrs Resolution Time,Storage,Oracle,Incident,Hard Disk Drive Failure,2.066667
63,4,24x7 4Hrs Resolution Time,Storage,Oracle,Incident,Hard Disk Drive Failure,2.266667
64,2,8x5 4Hrs Response Time,Switch,HPE,Incident,Network Adapter Failure,4.350000
65,2,8x5 NBD Response Time,Server,HPE,Incident,Network Adapter Failure,109.333333
66,3,24x7 4Hrs Resolution Time,Server,Oracle,Incident,Network Adapter Failure,479.716667


# Convert dataframe to tensorflow dataset

In [19]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>


In [20]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    # prob = tf.nn.sigmoid(predResult) # no need to convert to Signmoid
    # print(prob)
    # pred_seveirty_id= "critical" if _class==1 else "normal"
    _class= 1 if predResult[0]>=0.5 else 0  
    predServerityIDList.append(_class) #0=normal , 1=critical
    print(f"{predResult} : {_class}")

dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_binary_severity"])   


1/1 [==============================] - 0s 78ms/step
[0.5933329] : 1
[0.61333287] : 1
[0.23999985] : 0
[0.8733326] : 1
[0.6733328] : 1
[0.7033328] : 1
[0.4766663] : 0
[0.84333265] : 1
[0.83333266] : 1
[0.5599996] : 1
[0.443333] : 0
[0.99666584] : 1
[0.3866664] : 0
[0.3866664] : 0
[0.99999917] : 1
[0.99999917] : 1
[0.40666637] : 0
[0.5733329] : 1
[0.95333254] : 1
[0.16333325] : 0
[0.5733329] : 1
[0.99999917] : 1
[0.99999917] : 1
[0.7699994] : 1
[0.47999963] : 0
[0.19999988] : 0
[0.34999976] : 0
[0.38333306] : 0
[0.01333333] : 0
[0.38999972] : 0
[0.] : 0
[0.48999962] : 0
[0.00666667] : 0
[0.] : 0
[0.] : 0
[0.00333333] : 0
[0.] : 0
[0.00333333] : 0
[0.5733329] : 1
[0.6066662] : 1
[0.31999978] : 0
[0.] : 0
[0.] : 0
[0.] : 0
[0.00666667] : 0
[0.] : 0
[0.] : 0
[0.] : 0
[0.00666667] : 0
[0.02] : 0
[0.02] : 0
[0.02] : 0
[0.5199996] : 1
[0.8633326] : 1
[0.03333334] : 0
[0.06333334] : 0
[0.9033326] : 1
[0.99999917] : 1
[0.99999917] : 1
[0.9466659] : 1
[0.26333317] : 0
[0.5599996] : 1
[0.17333324]

2023-11-12 06:33:36.275465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [67]
	 [[{{node Placeholder/_2}}]]


# Map severity_id to label for actual value.
# Merge predicted value to main dataframe

In [21]:
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

dfPred
df=pd.concat([df,dfPred],axis=1)
# df

# Evaluate model and Show Metric Report

In [22]:
if is_evaluation:
    from sklearn.metrics import confusion_matrix,classification_report
    className=list(set().union(list(df['pred_binary_severity'].unique()),list(df['label_binary_severity'].unique())))
    print(className)
    actualClass=[  f'actual-{x}' for x in  className]
    predictedlClass=[  f'pred-{x}' for x in className]
    y_true=list(df['label_binary_severity'])
    y_pred=list(df['pred_binary_severity'])
    cnf_matrix = confusion_matrix(y_true,y_pred)
    cnf_matrix

    # #index=actual , column=prediction
    cm_df = pd.DataFrame(cnf_matrix,
                         index = actualClass, 
                         columns = predictedlClass)
    print(cm_df)
    print(classification_report(y_true, y_pred, labels=className))

[0, 1]
          pred-0  pred-1
actual-0      28       6
actual-1      10      23
              precision    recall  f1-score   support

           0       0.74      0.82      0.78        34
           1       0.79      0.70      0.74        33

    accuracy                           0.76        67
   macro avg       0.76      0.76      0.76        67
weighted avg       0.76      0.76      0.76        67



# Transform data for Writing Prediction Result to BQ

In [23]:
df=df[['id','label_binary_severity','pred_binary_severity']]
df['prediction_item_date']=today
df['prediction_datetime']=datetime.now()
df['model_version']=model_version
df

,id,label_binary_severity,pred_binary_severity,prediction_item_date,prediction_datetime,model_version
0,3951,1,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
1,3941,1,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
2,3891,1,0,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
3,3889,1,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
4,3851,0,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
...,...,...,...,...,...,...
62,3933,0,0,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
63,3931,0,0,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
64,3946,1,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23
65,3938,1,1,2023-11-12,2023-11-12 06:33:59.190116,demo2_binary_rf_tf_model/_demo_tNov23


# Load ata to BQ

In [34]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        if all_prediction==1: 
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
        else:
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  67 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'